<a href="https://colab.research.google.com/github/nvhien1961999/Covid19_auto_detecting_vision_transformer_model/blob/main/train_ViT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_folder = "/content/drive/MyDrive/ViT_3" + "/"

In [ ]:
!pip install split-folders transformers

     |████████████████████████████████| 3.3 MB 5.3 MB/s 
     |████████████████████████████████| 61 kB 499 kB/s 
     |████████████████████████████████| 596 kB 39.2 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 3.3 MB 31.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from torch import nn, cuda
from torch.utils import data
from sklearn import metrics
from transformers import ViTFeatureExtractor, ViTModel, AdamW
from tqdm.auto import tqdm
from torchvision import transforms

import os
import time
import splitfolders
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

device = torch.device("cuda" if cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# splitfolders.ratio(
#     input="/content/drive/MyDrive/dataset/dataset",
#     output="/content/drive/MyDrive/dataset/splitted_dataset",
#     seed=42,
#     ratio=(0.5, 0.25, 0.25),
# )

In [ ]:
vit_type = "google/vit-base-patch16-224-in21k"

vit_fe = ViTFeatureExtractor.from_pretrained(vit_type)
vit_model = ViTModel.from_pretrained(vit_type)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

In [ ]:
class ViTClassifier(nn.Module):
    def __init__(self, vit_model):
        super(ViTClassifier, self).__init__()

        self.vit = vit_model

        self.classifier = nn.Sequential(nn.Dropout(), nn.Linear(768, 3), nn.Softmax(1))

    def forward(self, x):
        out = self.vit(x).pooler_output
        return self.classifier(out)

In [ ]:
transform = torchvision.transforms.Compose(
    [
        lambda x: vit_fe(x, return_tensors="pt")["pixel_values"],
        lambda x: x.to(device).squeeze(0),
    ]
)

batch_size = 32
lr = 1e-3
max_epochs = 20
max_patience_scheduler = 2
max_patience_epoch = 5

train_dataset = torchvision.datasets.ImageFolder(
    "/content/drive/MyDrive/dataset/splitted_dataset/train", transform=transform
)
val_dataset = torchvision.datasets.ImageFolder(
    "/content/drive/MyDrive/dataset/splitted_dataset/val", transform=transform
)

trainloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = data.DataLoader(val_dataset, batch_size=batch_size)

vit_classifier = ViTClassifier(vit_model).to(device)
optimizer = AdamW(vit_classifier.parameters(), lr)

# checkpoint = torch.load(root_folder + "best_model.pt")
# vit_classifier.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

criterion = nn.CrossEntropyLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=max_patience_scheduler
)

In [ ]:
def write_file(file, value):
  file = root_folder + file
  f = open(file, "w")
  f.write(str(value))
  f.close()

def read_file(file, temp_list):
  file = root_folder + file
  with open(file) as f:
    for line in f:
        arr = line.split(",")
        for e in arr:
          v = e.split("[")[-1]
          v = v.split("]")[0]
          temp_list.append(float(v))

In [ ]:
accuracy_list = []
loss_list = []
f1_list = []
val_accuracy_list = []
val_loss_list = []
val_f1_list = []
training_time = 0

read_file("accuracy.txt",accuracy_list)
read_file("loss.txt",loss_list)
read_file("f1.txt",f1_list)
read_file("val_accuracy.txt",val_accuracy_list)
read_file("val_loss.txt",val_loss_list)
read_file("val_f1.txt",val_f1_list)
with open(root_folder + "training_time.txt") as f:
    for line in f:
        arr = line.split(",")
        for e in arr:
          v = e.split("[")[-1]
          v = v.split("]")[0]
          training_time = float(v)

In [ ]:
print("The number of item in loss list is " + str(len(loss_list)))

The number of item in loss list is 0


In [ ]:
print("The number of item in val_loss list is " + str(len(val_loss_list)))

The number of item in val_loss list is 0


In [ ]:
training_time

0.0

In [ ]:
metric_values = []
patience = 0

best_loss = None

# best_loss = checkpoint['loss']
# best_weight = vit_classifier.state_dict()
# for i in range(2,max_patience_epoch+2):
#     if(val_loss_list[-i] <= val_loss_list[-1]):
#         patience += 1
#     else:
#         break
# if patience > max_patience_epoch:
#     exit()

start = time.time()

for epoch in range(1, max_epochs + 1):
    with tqdm(
        total=len(trainloader),
        desc=f"Epoch {epoch}/{max_epochs}",
        unit="batch",
        position=0,
        leave=True,
    ) as pbar:
        true_target_train, pred_target_train = [], []
        losses_train = []
        vit_classifier.train()
        for x, y in trainloader:
            optimizer.zero_grad()
            y_pred = vit_classifier(x).cpu()
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            losses_train.append(loss.item())
            true_target_train.extend(y.tolist())
            pred_target_train.extend(y_pred.argmax(1).tolist())
            npArray = np.stack(losses_train, axis=0)
            pbar.set_postfix(
                {
                    "loss": f"{torch.tensor(npArray).mean().item():.4f}",
                    "accuracy": f"{metrics.accuracy_score(true_target_train, pred_target_train):.4f}",
                    "f1_score": f'{metrics.f1_score(true_target_train, pred_target_train, average="macro"):.4f}',
                }
            )
            pbar.update(1)
        true_target_val, pred_target_val = [], []
        losses_val = []
        vit_classifier.eval()
        for x, y in valloader:
            with torch.no_grad():
                y_pred = vit_classifier(x).cpu()
                loss = criterion(y_pred, y)
                losses_val.append(loss.item())
                true_target_val.extend(y.tolist())
                pred_target_val.extend(y_pred.argmax(1).tolist())
                npArray2 = np.stack(losses_train, axis=0)
                npArray3 = np.stack(losses_val, axis=0)
                pbar.set_postfix(
                    {
                        "loss": f"{torch.tensor(npArray2).mean().item():.4f}",
                        "accuracy": f"{metrics.accuracy_score(true_target_train, pred_target_train):.4f}",
                        "f1_score": f'{metrics.f1_score(true_target_train, pred_target_train, average="macro"):.4f}',
                        "val_loss": f"{torch.tensor(npArray3).mean().item():.4f}",
                        "val_accuracy": f"{metrics.accuracy_score(true_target_val, pred_target_val):.4f}",
                        "val_f1_score": f'{metrics.f1_score(true_target_val, pred_target_val, average="macro"):.4f}',
                    }
                )
        npArray4 = np.stack(losses_train, axis=0)
        npArray5 = np.stack(losses_val, axis=0)
        metric_values.append(
            {
                "loss": torch.tensor(npArray4).mean().item(),
                "accuracy": metrics.accuracy_score(
                    true_target_train, pred_target_train
                ),
                "f1_score": metrics.f1_score(
                    true_target_train, pred_target_train, average="macro"
                ),
                "val_loss": torch.tensor(npArray5).mean().item(),
                "val_accuracy": metrics.accuracy_score(
                    true_target_val, pred_target_val
                ),
                "val_f1_score": metrics.f1_score(
                    true_target_val, pred_target_val, average="macro"
                ),
            }
        )

        cur_acc = float(metric_values[-1]["accuracy"])
        cur_loss = float(metric_values[-1]["loss"])
        cur_f1 = float(metric_values[-1]["f1_score"])
        cur_val_acc = float(metric_values[-1]["val_accuracy"])
        cur_val_loss = float(metric_values[-1]["val_loss"])
        cur_val_f1 = float(metric_values[-1]["val_f1_score"])

        accuracy_list.append(cur_acc)
        loss_list.append(cur_loss)
        f1_list.append(cur_f1)
        val_accuracy_list.append(cur_val_acc)
        val_loss_list.append(cur_val_loss)
        val_f1_list.append(cur_val_f1)
        write_file("accuracy.txt", accuracy_list)
        write_file("loss.txt", loss_list)
        write_file("f1.txt", f1_list)
        write_file("val_accuracy.txt", val_accuracy_list)
        write_file("val_loss.txt", val_loss_list)
        write_file("val_f1.txt", val_f1_list)

        if best_loss is None or best_loss > metric_values[-1]["val_loss"]:
            best_weight = vit_classifier.state_dict()
            best_loss = metric_values[-1]["val_loss"]
            torch.save({
            'model_state_dict': vit_classifier.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': len(loss_list),
            'loss': metric_values[-1]["val_loss"],
            'accuracy': metric_values[-1]["val_accuracy"],
            'f1_score': metric_values[-1]["val_f1_score"]
            }, root_folder + "best_model.pt")
        else:
            patience += 1
        vit_classifier.load_state_dict(best_weight)

        scheduler.step(metric_values[-1]["val_loss"]) 
        end = time.time()
        f = open(root_folder + "training_time.txt", "w")
        f.write(str(end-start+training_time))
        f.close()

        if patience > max_patience_epoch:
            break

Epoch 1/20:   0%|          | 0/324 [00:00<?, ?batch/s]

Epoch 2/20:   0%|          | 0/324 [00:00<?, ?batch/s]

Epoch 3/20:   0%|          | 0/324 [00:00<?, ?batch/s]

Epoch 4/20:   0%|          | 0/324 [00:00<?, ?batch/s]

Epoch 5/20:   0%|          | 0/324 [00:00<?, ?batch/s]

Epoch 6/20:   0%|          | 0/324 [00:00<?, ?batch/s]

Epoch 7/20:   0%|          | 0/324 [00:00<?, ?batch/s]

Epoch 8/20:   0%|          | 0/324 [00:00<?, ?batch/s]

In [ ]:
patience

6